In [1]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 75%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [1]:
# TBD 1 : logger 추가
# TBD 2: flask github 참고, method, class, 파일의 맨 윗단 마다 pydoc 형식으로 달기
# TBD 3: 축약어를 자제할것 (특히 변수)

# tensorflow Module
import tensorflow as tf
from tensorflow.keras import backend as keras_backend
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import losses

# python basic Module
import os
import sys
import types
from datetime import datetime
from shutil import copy
from pickle import dump, load

# math, image, plot Module
import numpy as np
import cv2
import matplotlib.pyplot as plt  # TBD

# email Module
import smtplib
from email.mime.text import MIMEText
from email.header import Header

from data_loader.medical_segmentation_data_loader_v1 import DataLoader

from gan_module.model import build_generator, build_discriminator
from gan_module.draw_images import ImageDrawer
from gan_module.custom_loss import f1_loss_for_training, f1_score

# set GPU memory growth allocation
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

"""
if you don't have nvidia-gpu, try plaidml! but it will works tensorflow 1.x.x
# pip install -U plaidml-keras
# plaidml-setup
"""
# use_plaidml = False
# if use_plaidml :
#     import plaidml.keras
#     plaidml.keras.install_backend()
#     os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"


# class CaptchaDiscriminator
# captcha_generator = CaptchaGenerator()
# CaptchaGeneratorTrainer(captcha_generator, discriminator)
# class CaptchaGeneratorTrainer():
# class LearningRate():
# class Smtp()
# class Shape()
# class InputShape(Shape):
# class OutputShape(Shape):
# class Model():
#     def save():
#     def load():
# class Generator(Model) :
#     @override
#     def save():
#         super()
# class HyperParameter()


class Pix2PixSegmentation:
    def __init__(
        self,
        generator_power=32,
        discriminator_power=32,
        generator_learning_rate=1e-4,
        discriminator_learning_rate=1e-4,
        learning_rate_decay_rate_epoch=0.1,
        learning_rate_decay_rate_dynamic=0.1,
        find_init_epoch=150,
        find_error=False,
        temp_weights_path=".",
        draw_images=True,
        on_memory=True,
        test=False
    ):
        # smtp info
        self.smtp_host = "smtp.gmail.com"
        self.smtp_port = 465
        self.smtp_id = "rpa.manager0001@gmail.com"
        self.smtp_password = "!rpa.admin!23"
        self.smtp_to_addr = "tobeor3009@gmail.com"

        # Input shape
        self.img_rows = 512
        self.img_cols = 512
        self.input_channels = 3
        self.output_channels = 1
        self.input_img_shape = (
            self.img_rows, self.img_cols, self.input_channels)
        self.output_img_shape = (
            self.img_rows, self.img_cols, self.output_channels)
        # set parameter
        self.start_epoch = None
        self.history = [[], [], []]
        self.learning_rate_decay_rate_epoch = learning_rate_decay_rate_epoch
        self.learning_rate_decay_rate_dynamic = learning_rate_decay_rate_dynamic
        self.find_init_epoch = find_init_epoch
        self.find_error = find_error
        self.find_error_epoch = 30
        self.error_list = []
        self.temp_weights_path = temp_weights_path

        # Configure data loader
        self.dataset_name = "tumor"
        self.data_loader = DataLoader(
            dataset_name=self.dataset_name,
            img_res=(self.img_rows, self.img_cols),
            on_memory=on_memory, test=test
        )
        self.train_loaded_data, self.valid_loaded_data = self.data_loader.load_all()
        if test:
            self.train_loaded_data_len = 20
            self.valid_loaded_data_len = 20
        else:
            self.train_loaded_data_len = self.data_loader.train_data_length
            self.valid_loaded_data_len = self.data_loader.valid_data_length

        self.train_loaded_data_index = np.arange(self.train_loaded_data_len)
        self.valid_loaded_data_index = np.arange(self.valid_loaded_data_len)

        # Configure Image Drawer
        self.draw_images = draw_images
        self.image_drawer = ImageDrawer(
            dataset_name=self.dataset_name, data_loader=self.data_loader
        )
        # training parameters
        self.learning_schedule = [
            50,
            100,
            150,
            200,
            250,
            300,
            350,
            400,
            450,
            500,
            550
        ]
        self.discriminator_acc_previous = 0.5
        self.generator_loss_min = 100
        self.generator_loss_previous = 100
        self.generator_loss_max_previous = 1000
        self.generator_loss_max_min = 1000
        self.generator_loss_min_min = 1000
        self.weight_save_stack = False
        self.weight_stagnant_stack = 0
        self.look_on_for_generator_min_loss = False
        self.learning_rate_decay_dynamic = 0
        self.training_end_stack = 0
        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 2)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.generator_power = generator_power
        self.discriminator_power = discriminator_power
        self.generator_learning_rate = generator_learning_rate
        self.discriminator_learning_rate = discriminator_learning_rate
        generator_optimizer = Nadam(self.generator_learning_rate)
        discriminator_optimizer = Nadam(self.discriminator_learning_rate)

        # layer Component
        self.kernel_initializer = RandomNormal(mean=0.0, stddev=0.02)

        # Build and compile the discriminator
        self.discriminator = build_discriminator(
            output_img_shape=self.output_img_shape,
            discriminator_power=self.discriminator_power,
            kernel_initializer=self.kernel_initializer,
        )
        # self.discriminator = self.build_discriminator()
        # 'mse' or tf.keras.losses.Huber() tf.keras.losses.LogCosh()
        self.discriminator.compile(
            loss=tf.keras.losses.LogCosh(),
            optimizer=discriminator_optimizer,
            metrics=["accuracy"],
        )

        # -------------------------
        # Construct Computational
        #   Graph of Generator
        # -------------------------

        # Build the generator
        self.generator = build_generator(
            input_img_shape=self.input_img_shape,
            output_channels=self.output_channels,
            generator_power=self.generator_power,
            kernel_initializer=self.kernel_initializer,
        )

        # Input images and their conditioning images
        original_img = Input(shape=self.input_img_shape)
        masked_img = Input(shape=self.output_img_shape)
        # generate image from original_img for target masked_img
        model_masked_img = self.generator(original_img)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        # Discriminators determines validity of translated images / condition pairs
        model_validity = self.discriminator(model_masked_img)
        # give score by
        # 1. how generator trick discriminator
        # 2. how generator's image same as real photo in pixel
        # 3. if you want change loss, see doc https://keras.io/api/losses/
        # 4. 'mse', 'mae', tf.keras.losses.LogCosh(),  tf.keras.losses.Huber()
        self.combined = Model(
            inputs=[original_img, masked_img],
            outputs=[model_validity, model_masked_img, model_masked_img],
        )
        self.combined.compile(
            loss=[
                tf.keras.losses.LogCosh(),
                tf.keras.losses.Huber(),
                f1_loss_for_training
            ],
            loss_weights=[1, 100, 0.5],
            optimizer=generator_optimizer
        )

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.now()

        # Adversarial loss ground truths
        self.training_end_stack = 0
        self.batch_size = batch_size
        valid_patch = np.ones((self.batch_size,) +
                              self.disc_patch, dtype=np.float32)
        fake_patch = np.zeros((self.batch_size,) +
                              self.disc_patch, dtype=np.float32)
        if self.start_epoch is None:
            self.start_epoch = 0
        for epoch in range(self.start_epoch, epochs):
            batch_i = 0
            discriminator_acces = []
            train_generator_loss = []
            generator_loss_max_in_epoch = 0
            generator_loss_min_in_epoch = 1000
            generator_current_learning_rate = self.learning_rate_scheduler(
                self.generator_learning_rate
                * (2 ** (self.learning_rate_decay_dynamic)),
                epoch,
            )
            discriminator_current_learning_rate = self.learning_rate_scheduler(
                self.discriminator_learning_rate
                * (2 ** (self.learning_rate_decay_dynamic)),
                epoch,
            )
            keras_backend.set_value(
                self.combined.optimizer.learning_rate, generator_current_learning_rate
            )
            keras_backend.set_value(
                self.discriminator.optimizer.learning_rate,
                discriminator_current_learning_rate,
            )
            # shffle data 10 epoch term
            if epoch % 10 == 0 and not isinstance(self.train_loaded_data, types.GeneratorType):
                np.random.shuffle(self.train_loaded_data_index)
                self.train_loaded_data = [
                    self.train_loaded_data[i][self.train_loaded_data_index] for i in range(2)
                ]
            if self.discriminator_acc_previous < 0.75 or epoch < 2:
                discriminator_learning = True
                print("discriminator_learning is True")
            else:
                discriminator_learning = False
                print("discriminator_learning is False")
                
            while batch_i + self.batch_size <= self.train_loaded_data_len:
                original_img = self.train_loaded_data[0][batch_i: batch_i +
                                                         self.batch_size]
                masked_img = self.train_loaded_data[1][batch_i: batch_i +
                                                       self.batch_size]

                # ---------------------
                #  Train Discriminator
                # ---------------------
                # Condition on B and generate a translated version
                model_masked_img = self.generator.predict_on_batch(
                    original_img)

                # forTest
                self.masked_img = masked_img
                self.original_img = original_img
                self.model_masked_img = model_masked_img
                self.valid_path = valid_patch
                self.fake_patch = fake_patch
                # Train the discriminators (target image = masked_img / generated_img = model_masked_img)
                if discriminator_learning:
                    self.discriminator.train_on_batch(masked_img, valid_patch)
                discriminator_loss = self.discriminator.test_on_batch(model_masked_img, fake_patch)

                # -----------------
                #  Train Generator
                # -----------------

                # Train the generators
                generator_loss = self.combined.train_on_batch(
                    [original_img, masked_img],
                    [valid_patch, masked_img, masked_img],
                )
                elapsed_time = datetime.now() - start_time
                if batch_i % sample_interval == 0:
                    # Plot the progress
                    print(
                        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s"
                        % (
                            epoch,
                            epochs,
                            batch_i,
                            self.train_loaded_data_len,
                            discriminator_loss[0],
                            100 * discriminator_loss[1],
                            generator_loss[0],
                            elapsed_time,
                        )
                    )

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0 and self.draw_images:
                    self.image_drawer.sample_images(
                        self.generator, epoch, batch_i)

                discriminator_acces.append(discriminator_loss[1])
                train_generator_loss.append(generator_loss[0])
                # loss 가 가장 높은 이미지를 저장 및 max_in_epoch 갱신
                if generator_loss[0] > generator_loss_max_in_epoch:
                    model_masked_img = self.generator.predict_on_batch(
                        original_img)
                    if self.draw_images:
                        self.image_drawer.draw_worst_and_best(
                            original_img,
                            model_masked_img,
                            masked_img,
                            epoch,
                            worst=True,
                        )
                    generator_loss_max_in_epoch = generator_loss[0]
                # loss 가 가장 낮은 이미지를 저장 및 max_in_epoch 갱신
                if generator_loss_min_in_epoch > generator_loss[0]:
                    model_masked_img = self.generator.predict_on_batch(
                        original_img)
                    if self.draw_images:
                        self.image_drawer.draw_worst_and_best(
                            original_img,
                            model_masked_img,
                            masked_img,
                            epoch,
                            worst=False,
                        )
                    generator_loss_min_in_epoch = generator_loss[0]

                # 한 배치 끝
                batch_i += self.batch_size
            # training batch 사이클 끝
            self.history[0].append(discriminator_loss[0])
            self.history[1].append(generator_loss[0])
            self.history[2].append(100 * discriminator_loss[1])
            print(f"discriminator_acces : {str(np.mean(discriminator_acces))}")
            print(
                f"Mean generator_loss : {str(np.mean(train_generator_loss))}")
            print(f"Max generator_loss : {str(np.max(train_generator_loss))}")
            print(f"Min generator_loss : {str(np.min(train_generator_loss))}")
            print(
                f"generator loss decrease : {str(self.generator_loss_previous - np.mean(train_generator_loss))}"
            )
            print(
                f"Max generator loss decrease : {str(self.generator_loss_max_previous - np.max(train_generator_loss))}"
            )
            print(
                f"current lowest generator loss : {str(self.generator_loss_min)}")

            # set look_on_for_generator_min_loss property if min loss decrease:
            if (
                self.generator_loss_min_min > generator_loss_min_in_epoch
                and self.generator_loss_max_min * 2 > generator_loss_max_in_epoch
            ):
                self.look_on_for_generator_min_loss = True
            else:
                self.look_on_for_generator_min_loss = False
            # rollback if loss not converge
            if np.mean(train_generator_loss) / self.generator_loss_min < 1.02:
                if self.generator_loss_min > np.mean(train_generator_loss):
                    # 학습중일때 진전이 너무 더디다면 learning_rate 을 조정하는 스택을 추가.
                    # loss의 감소량이 너무 적을때
                    if np.mean(train_generator_loss) / self.generator_loss_min > 0.97:
                        self.learning_rate_decay_dynamic -= (
                            self.learning_rate_decay_rate_dynamic / 10
                        )
                        print(
                            "increase Learning rate(learning_rate_increaseStack ="
                            + str(round(self.learning_rate_decay_dynamic, 3))
                            + ")"
                            + "ratio = ("
                            + str(np.mean(train_generator_loss) /
                                  self.generator_loss_min)
                            + ")"
                        )
                    elif np.mean(train_generator_loss) / self.generator_loss_min < 0.85:
                        # loss의 감소량이 너무 클때
                        self.learning_rate_decay_dynamic -= (
                            self.learning_rate_decay_rate_dynamic / 10
                        )
                        print(
                            "decrease Learning rate(learning_rate_increaseStack ="
                            + str(round(self.learning_rate_decay_dynamic, 3))
                            + ")"
                            + "ratio = ("
                            + str(np.mean(train_generator_loss) /
                                  self.generator_loss_min)
                            + ")"
                        )
                    self.generator_loss_min = np.mean(train_generator_loss)
                    self.generator_loss_max_min = generator_loss_max_in_epoch
                    self.generator_loss_min_min = generator_loss_min_in_epoch
                    self.weight_save_stack = True
                    self.save_study_info()
                    print("save weights")
                else:
                    # loss가 약간 증가 했을때 (정체 가능성)
                    self.learning_rate_decay_dynamic -= (
                        self.learning_rate_decay_rate_dynamic / 2
                    )
                    print(
                        "decrease Learning rate(learning_rate_increaseStack ="
                        + str(round(self.learning_rate_decay_dynamic, 3))
                        + ")"
                        + "ratio = ("
                        + str(np.mean(train_generator_loss) /
                              self.generator_loss_min)
                        + ")"
                    )
            else:
                if self.look_on_for_generator_min_loss:
                    print("min_loss is decreased. watch")
                else:
                    print("loss decreasing")
                    self.learning_rate_decay_dynamic -= (
                        self.learning_rate_decay_rate_dynamic
                    )
                    print(
                        "decrease Learning rate(learning_rate_increaseStack ="
                        + str(round(self.learning_rate_decay_dynamic, 3))
                        + ")"
                    )
                    self.load_best_weights()
            # set look_on_for_generator_min_loss property False anyway because of confusing training
            self.look_on_for_generator_min_loss = False
            # previous generator_loss 갱신
            self.generator_loss_previous = np.mean(train_generator_loss)
            self.generator_loss_max_previous = generator_loss_max_in_epoch

            if epoch >= 100 and self.weight_save_stack:
                copy(
                    "generator.h5",
                    "./generator_weights/generator_"
                    + str(round(self.generator_loss_min, 5))
                    + "_"
                    + str(round(self.generator_loss_max_min, 5))
                    + ".h5",
                )
                self.weight_save_stack = False

            self.discriminator_acc_previous = np.mean(discriminator_acces)
            
            valid_f1_loss_list = []
            valid_f1_score_list = []
            valid_predict_mini_batch_size = 1
            for index in range(0, self.valid_loaded_data_len, valid_predict_mini_batch_size):
                
                valid_original_img = self.valid_loaded_data[0][index:index+valid_predict_mini_batch_size]
                valid_masked_img = self.valid_loaded_data[1][index:index+valid_predict_mini_batch_size]
                valid_model_masked_img = self.generator.predict_on_batch(
                self.valid_loaded_data[0][index:index+valid_predict_mini_batch_size])
                
                valid_f1_loss = f1_loss_for_training(valid_masked_img, np.squeeze(valid_model_masked_img))
                valid_f1_score = f1_score(valid_masked_img, np.squeeze(valid_model_masked_img))
                valid_f1_loss_list.append(valid_f1_loss)
                valid_f1_score_list.append(valid_f1_score)
            print(f"valid_f1_loss : {np.mean(valid_f1_loss_list)}")
            print(f"valid_f1_score : {np.mean(valid_f1_score_list)}")
    def learning_rate_scheduler(self, learning_rate, epoch):

        for step in range(0, len(self.learning_schedule)):
            if epoch < self.learning_schedule[step]:
                break
        new_learning_rate = learning_rate * (
            self.learning_rate_decay_rate_epoch ** (step)
        )
        return new_learning_rate

    def get_info_folderPath(self):
        return (
            str(round(self.generator_loss_min, 5))
            + "_"
            + str(round(self.generator_loss_max_min, 5))
            + "_"
            + str(round(self.learning_rate_decay_dynamic, 3))
        )

    def save_study_info(self, path=None):

        if path == None:
            path = self.temp_weights_path

        generator_weigth_path = os.path.join(path, "generator.h5")
        discriminator_weigth_path = os.path.join(path, "discriminator.h5")
        combined_weigth_path = os.path.join(path, "combined.h5")

        self.generator.save_weights(generator_weigth_path)
        self.discriminator.save_weights(discriminator_weigth_path)
        self.combined.save_weights(combined_weigth_path)

        study_info = {}
        study_info["start_epoch"] = self.start_epoch
        study_info["generator_loss_min"] = self.generator_loss_min
        study_info["generator_loss_max_min"] = self.generator_loss_max_min
        study_info["generator_loss_min_min"] = self.generator_loss_min_min
        study_info["learning_rate_decay_dynamic"] = self.learning_rate_decay_dynamic

        file = open(path + "/study_info.pkl", "wb")
        dump(study_info, file)
        file.close()

    def load_best_weights(self):
        self.generator.load_weights(self.temp_weights_path + "/generator.h5")
        self.discriminator.load_weights(
            self.temp_weights_path + "/discriminator.h5")
        self.combined.load_weights(self.temp_weights_path + "/combined.h5")

    def load_study_info(self):

        self.generator.load_weights("generator.h5")
        self.discriminator.load_weights("discriminator.h5")
        self.combined.load_weights("combined.h5")

        if os.path.isfile("study_info.pkl"):
            file = open("study_info.pkl", "rb")
            study_info = load(file)
            file.close()
            self.start_epoch = study_info["start_epoch"]
            self.generator_loss_min = study_info["generator_loss_min"]
            self.generator_loss_max_min = study_info["generator_loss_max_min"]
            self.generator_loss_min_min = study_info["generator_loss_min_min"]
            self.learning_rate_decay_dynamic = study_info["learning_rate_decay_dynamic"]
        else:
            print("No info pkl file!")


In [2]:
gan = Pix2PixSegmentation(generator_power=4, discriminator_power=12, generator_learning_rate=1e-4,discriminator_learning_rate=1e-3,
                          learning_rate_decay_rate_epoch = 0.5, learning_rate_decay_rate_dynamic = 0.1, test=False)

In [3]:
#gan.load_study_info()

In [4]:
#gan.find_error = True
#gan.find_error_epoch = 5

#gan.start_epoch = 0
gan.train(epochs=575, batch_size=1, sample_interval=3400)

discriminator_learning is True
[Epoch 0/575] [Batch 0/4700] [D loss: 0.000750, acc: 100%] [G loss: 13.627122] time: 0:00:38.041497
[Epoch 0/575] [Batch 3400/4700] [D loss: 0.445724, acc:   0%] [G loss: 1.206365] time: 0:07:49.254500
discriminator_acces : 0.05958290911735372
Mean generator_loss : 4.173301646899669
Max generator_loss : 23.857711791992188
Min generator_loss : 0.6144253015518188
generator loss decrease : 95.82669835310033
Max generator loss decrease : 976.1422882080078
current lowest generator loss : 100
decrease Learning rate(learning_rate_increaseStack =-0.01)ratio = (0.04173301646899669)
save weights
valid_f1_loss : 0.9176703691482544
valid_f1_score : 0.10863261669874191
discriminator_learning is True
[Epoch 1/575] [Batch 0/4700] [D loss: 0.460156, acc:   0%] [G loss: 3.694846] time: 0:10:51.040999
[Epoch 1/575] [Batch 3400/4700] [D loss: 0.432449, acc:   0%] [G loss: 1.327698] time: 0:17:21.453501
discriminator_acces : 0.001961592004654255
Mean generator_loss : 2.75621

[Epoch 12/575] [Batch 0/4700] [D loss: 0.433663, acc:   0%] [G loss: 4.434058] time: 1:55:12.963044
[Epoch 12/575] [Batch 3400/4700] [D loss: 0.433674, acc:   0%] [G loss: 1.906049] time: 2:01:42.173042
discriminator_acces : 0.0
Mean generator_loss : 2.0477093076705932
Max generator_loss : 12.702021598815918
Min generator_loss : 0.5082948803901672
generator loss decrease : 0.021874393313489282
Max generator loss decrease : 0.06774330139160156
current lowest generator loss : 2.0695837009840825
increase Learning rate(learning_rate_increaseStack =-0.14)ratio = (0.9894305346031242)
save weights
valid_f1_loss : 0.9422909021377563
valid_f1_score : 0.04919610172510147
discriminator_learning is True
[Epoch 13/575] [Batch 0/4700] [D loss: 0.433675, acc:   0%] [G loss: 4.162458] time: 2:04:42.811044
[Epoch 13/575] [Batch 3400/4700] [D loss: 0.433663, acc:   0%] [G loss: 1.934363] time: 2:11:11.373059
discriminator_acces : 0.0
Mean generator_loss : 2.0216833241442416
Max generator_loss : 13.55250

save weights
valid_f1_loss : 0.9159131646156311
valid_f1_score : 0.08110971748828888
discriminator_learning is True
[Epoch 24/575] [Batch 0/4700] [D loss: 0.433683, acc:   0%] [G loss: 1.368947] time: 3:49:04.129042
[Epoch 24/575] [Batch 3400/4700] [D loss: 0.433702, acc:   0%] [G loss: 1.534021] time: 3:55:34.010044
discriminator_acces : 0.0
Mean generator_loss : 1.8490276237497938
Max generator_loss : 10.041635513305664
Min generator_loss : 0.5147377848625183
generator loss decrease : 0.013326790751294837
Max generator loss decrease : 0.24951553344726562
current lowest generator loss : 1.8623544145010886
increase Learning rate(learning_rate_increaseStack =-0.3)ratio = (0.992844116754831)
save weights
valid_f1_loss : 0.9135108590126038
valid_f1_score : 0.08440262824296951
discriminator_learning is True
[Epoch 25/575] [Batch 0/4700] [D loss: 0.433699, acc:   0%] [G loss: 1.275425] time: 3:58:34.991545
[Epoch 25/575] [Batch 3400/4700] [D loss: 0.433688, acc:   0%] [G loss: 1.534201] tim

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\gr300\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-de394a4c74c7>", line 5, in <module>
    gan.train(epochs=575, batch_size=1, sample_interval=3400)
  File "<ipython-input-1-ab4db209ac03>", line 316, in train
    generator_loss = self.combined.train_on_batch(
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py", line 1698, in train_on_batch
    self.reset_metrics()
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py", line 1637, in reset_metrics
    m.reset_states()
  File "C:\Users\gr300\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\metrics.py", line 247, in reset_states
    K.batch_set_value([(v, 0) for v in self.variables])
  File "C:\Users\gr300\AppData\Roaming\Python\Pyt

TypeError: object of type 'NoneType' has no len()

In [ ]:
gan.save_study_info()

In [ ]:
#gan.find_error = True
#gan.find_error_epoch = 5
gan.start_epoch = 0
gan.train(epochs=575, batch_size=4, sample_interval=3400)

In [ ]:
gan.save_study_info()

In [ ]:
predicted_img = gan.generator.predict_on_batch(original_img)

In [ ]:
a = list(range(3))

print(a[1,3])

In [ ]:
gan.discriminator.test_on_batch([original_img, predicted_img], gan.valid_path)

In [ ]:
gan.combined.train_on_batch(
    [original_img, predicted_img],
    [gan.valid_path, masked_img],
)

In [ ]:
gan.loaded_data_index

In [ ]:
print(gan.loaded_data[1][0])

In [ ]:
gan.model_masked_img[:,:,:,0][0]

In [ ]:
cv2.cvtColor(gan.model_masked_img[:,:,:,0][0], cv2.COLOR_GRAY2RGB).shape

In [ ]:
gan.load_study_info()

In [ ]:
print(gan.original_img.shape)
print(gan.model_masked_img.shape)
print(gan.masked_img.shape)

In [ ]:
temp_origin = ((gan.original_img[0]+1) * 127.5).astype('uint8')
temp_masked = ((gan.masked_img[0]+1) * 127.5).astype('uint8')

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(temp_origin)
plt.show()

In [ ]:
plt.imshow(gan.loaded_data[1][0])
plt.show()

In [ ]:
from glob import glob

images = glob("C:\\Users\\gr300\\Desktop\\Works\의료데이터\\Level_0_512_random_Split\\FOLD_1\\wo_SN\\slide-2020-04-22T09-53-31-R3-S20\\image_MONO_random\\*")

print(len(images))

In [ ]:
def my_gen